# A* algorithm

In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from heapq import heappush, heappop
import time

In [2]:
# Read data from the CSV file into a Pandas DataFrame
df = pd.read_csv("~/Downloads/Flight_Data.csv")

In [234]:
# Create a dictionary to store airport locatoins
Airport_loc = {}

# Iterate through the DataFrame to populate the Airport_loc dictionary
for ind in df.index:
    # For each airport, store its latitude, longitude, and altitude in a list
    Airport_loc[df['SourceAirport'][ind]] = [
        df['SourceAirport_Latitude'][ind],
        df['SourceAirport_Longitude'][ind],
        df['SourceAirport_Altitude'][ind]
    ]
    Airport_loc[df['DestinationAirport'][ind]] = [
        df['DestinationAirport_Latitude'][ind],
        df['DestinationAirport_Longitude'][ind],
        df['DestinationAirport'][ind]
    ]

In [235]:
# Create a dictionary to represent a flight graph
graph = {}

# Populate the flight graph using data from the DataFrame
for ind in df.index:
    if df['SourceAirport'][ind] not in graph.keys():
        # If the source airport is not in the graph, create an empty list for it
        graph[df['SourceAirport'][ind]] = []

    # Append destination airport and flight details to the source airport's list
    graph[df['SourceAirport'][ind]].append(
        [
            df['DestinationAirport'][ind],
            [df['Distance'][ind], df['FlyTime'][ind], df['Price'][ind]]
        ]
    )

In [236]:
input_string = input()

# Split the input string into two parts using the hyphen as the delimiter
start_airport, goal_airport = input_string.split(" - ")

In [237]:
start_time = time.time()

In [238]:
def geuristic(flight_detail):
    return 1*flight_detail[0] + 0*flight_detail[1] + 0*flight_detail[2]

def heuristic(cur_node, goal):
    # Euclidean
    x = abs (cur_node[0] - goal[0])
    y = abs (cur_node[1] - goal[1])
    #z = abs (cur_node[2] - goal[2])
    return ((x*x) + (y*y)) ** 0.5

openList = []
closedList = {}
details = {}

# Add values of start airport to the openList and
# set values of g, h, f and parent for start airport in this order
details[start_airport] = (0, 1, 1, -1)
heappush (openList, (0, start_airport))

In [239]:
# Continue looping until the heap is not empty
while openList:
    # Get and remove the node with largest f
    cur_node = heappop(openList)
    closedList[cur_node[1]] = True

    all_destinations = {}
    found_destination = False
    # Store the g, h and f of the all possible destination airports with minimum f
    for destination in graph[cur_node[1]]:
        if destination[0] not in closedList.keys():
            # Calculating g, h and f for current destination
            destination_g = details[cur_node[1]][0] + geuristic(destination[1])
            destination_h = heuristic(Airport_loc[destination[0]], Airport_loc[goal_airport])
            destination_f = destination_g + destination_h

            # If the current destination was never added before or f is larger than last flight replace this instead
            if destination[0] not in all_destinations.keys() or all_destinations[destination[0]][2] < destination_f:
                all_destinations[destination[0]] = (destination_g, destination_h, destination_f, cur_node[1])
    
    for destination in all_destinations.keys():
        if destination == goal_airport:
            print("Found The Destination!!\n")
            #print(details[cur_node[1]][0] + all_destinations[destination][0])
            details[goal_airport] = all_destinations[destination]
            #print(details[goal_airport])
            found_destination = True
            break

        if destination not in details.keys() or details[destination][2] > all_destinations[destination][2]:
            heappush (openList, (all_destinations[destination][2], destination))
            details[destination] = all_destinations[destination]
    
    if found_destination:
        break

path_stack = []
cur_node = goal_airport
while cur_node != -1:
    path_stack.append (cur_node)
    cur_node = details[cur_node][3]

end_time = time.time()

print("A* Algorithm")
print("Execution Time: ", end_time-start_time, "seconds")
while len(path_stack):
    print(path_stack.pop())

if not found_destination:
    print("THERE IS NO PATH BETWEEN LAST AIRPORT AND GOAL AIRPORT!")

Found The Destination!!

A* Algorithm
Execution Time:  0.026277780532836914  seconds
Imam Khomeini International Airport
Atatürk International Airport
John F Kennedy International Airport
Raleigh Durham International Airport


## Dijkstra

In [46]:
class airport_information:
    def __init__(self, city, country, latitude, longitude, altitude):
        self.city = city
        self.country = country
        self.latitude = latitude
        self.longitude = longitude
        self.altitude = altitude

In [60]:
airports = set()
airports_index = dict()
airports_information = np.empty(1000, dtype=airport_information)

ind = 0
for i in df.index:
    airport = df["SourceAirport"][i]
    if airport not in airports:
        airports.add(airport)
        airports_index[airport] = ind
        inf = airport_information(df["SourceAirport_City"][i], df["SourceAirport_Country"][i],
                                   df["SourceAirport_Latitude"][i], df["SourceAirport_Longitude"][i],
                                   df["SourceAirport_Altitude"][i])
        airports_information[ind] = inf
        ind += 1
        
    airport = df["DestinationAirport"][i]
    if airport not in airports:
        airports.add(airport)
        airports_index[airport] = ind
        inf = airport_information(df["DestinationAirport_City"][i], df["DestinationAirport_Country"][i],
                                   df["DestinationAirport_Latitude"][i], df["DestinationAirport_Longitude"][i],
                                   df["DestinationAirport_Altitude"][i])
        airports_information[ind] = inf
        ind += 1

number_of_airports = len(airports)
print(number_of_airports)

759


In [66]:
class flight_information:
    def __init__(self, source_airport, destination_airport, distance, flytime, price):
        self.source_airport = source_airport
        self.destination_airport = destination_airport
        self.distance = distance 
        self.fiytime = flytime
        self.price = price
        
class flights_list:
    def __init__(self):
        self.flights = []
    def add_flight(self, flight):
        self.flights.append(flight)

In [79]:
flights_information = np.empty(1000, dtype=flights_list)
for i in range(number_of_airports):
    flights_information[i] = flights_list()
num = 0
for i in df.index:
    index_source_airport = air_ports_index[df["SourceAirport"][i]]
    index_destination_airport = air_ports_index[df["DestinationAirport"][i]]
    distance = df["Distance"][i]
    flytime = df["FlyTime"][i]
    price = df["Price"][i]
    inf = flight_information(index_source_airport, index_destination_airport, distance, flytime, price)
    flights_information[index_source_airport].add_flight(inf)
    num +

print(num)

array([<__main__.flights_list object at 0x7f573cf7ec90>,
       <__main__.flights_list object at 0x7f573b333850>, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None